# Generate  circles using evolutionary algorithm

The code is based on 
https://github.com/tjwei/play_nsfw, 
which is based on https://scturtle.me/posts/2014-04-18-ga.html 

It depends on DEAP https://deap.readthedocs.io/en/master/

In [ ]:
import multiprocessing
from deap import base, creator, tools, algorithms
from PIL import Image, ImageDraw
from random import randint
import skimage

In [ ]:
SIZE_X, SIZE_Y = 256, 256
NUMBER_OF_TRIANGLES = 50
POPULATION = 50
NGEN = 4000
POLY = 3

In [ ]:
def gen_one_triangle():
    return (tuple([(randint(0, SIZE_X), randint(0, SIZE_Y)) for i in range(POLY)]),
            randint(0,255), randint(0,255), randint(0,255), randint(0,128))

In [ ]:
creator.create("Fitness", base.Fitness, weights=(1.0,))  # maximize fitness
creator.create("Individual", list, fitness=creator.Fitness)  # individual class

toolbox = base.Toolbox()
toolbox.register("attr", gen_one_triangle)  # the above function
toolbox.register("individual", tools.initRepeat,  # initialization of individual
                 creator.Individual, toolbox.attr, NUMBER_OF_TRIANGLES)
toolbox.register("population", tools.initRepeat,  # initialization of population
                 list, toolbox.individual)

In [ ]:
import numpy as np
target_im = Image.open('sample_images/800px-Meisje_met_de_parel.jpg').crop((0,100,800,900)).resize((256,256), Image.LANCZOS)
target_array = np.array(target_im, dtype='float')/255.
target_im

In [ ]:
def triangles_to_image(triangles):
    im = Image.new('RGB', (SIZE_X, SIZE_Y), (0, 0, 0))
    for tri in triangles:
        mask = Image.new('RGBA', (SIZE_X, SIZE_Y))
        draw = ImageDraw.Draw(mask)
        draw.polygon(tri[0], fill=tri[1:])
        im.paste(mask, mask=mask)
        del mask, draw
    return im

def evaluate(t2):
    output_im = triangles_to_image(t2)
    output_array = np.array(output_im, dtype='float')/255.
    score = skimage.measure.compare_psnr(output_array, target_array, 1.)
    #score = skimage.measure.compare_ssim(output_array, target_array, data_range=1., multichannel=True)
    return (score,)

In [ ]:
def mutate(triangles):
    e0 = triangles.fitness.values
    for i in range(10):
        tid = randint(0, NUMBER_OF_TRIANGLES - 1)
        oldt = triangles[tid]

        p = randint(-1, 2 * POLY + 4 - 1)
        if p == -1:
            tid2 = randint(0, NUMBER_OF_TRIANGLES - 1)
            triangles[tid], triangles[tid2] = triangles[tid2], oldt
        else:
            t = list(oldt)
            if p < 2 * POLY:
                points = list(t[0])
                pnt = list(points[p // 2])
                if p%2 == 0:
                    pnt[0] = randint(0, SIZE_X)
                else:
                    pnt[1] = randint(0, SIZE_Y)
                points[p // 2] = tuple(pnt)
                t[0] = tuple(points)
            else:
                p -= 2 * POLY - 1
                t[p] = randint(0, 255)

            triangles[tid] = tuple(t)
   
        if evaluate(triangles) > e0:
            break
        else:
            if p == -1:
                triangles[tid], triangles[tid2] = oldt, triangles[tid]
            else:
                triangles[tid] = oldt
    return [triangles]

In [ ]:
toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxTwoPoint)  # crossover
toolbox.register("mutate", mutate)  # mutation
toolbox.register("select", tools.selTournament, tournsize=3)

In [ ]:
from IPython.display import display, clear_output
import numpy as np
class ipyHOF(tools.HallOfFame):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.last_displayed_fitness = -1
    def insert(self, item):
        if item.fitness.values[0] >= self.last_displayed_fitness+0.001:
            self.last_displayed_fitness = item.fitness.values[0]
            clear_output(True)
            display(triangles_to_image(item))        
        super().insert(item)

In [ ]:
pop = toolbox.population(n=POPULATION)
hof = ipyHOF(3)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("std", np.std)
stats.register("max", np.max)
stats.register("avg", np.mean)
stats.register("min", np.min)

# For multiprocessing
pool = multiprocessing.Pool()
toolbox.register("map", pool.map)

In [ ]:
try:
    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.1, ngen=NGEN, 
                               stats=stats, halloffame=hof, verbose=True)
except KeyboardInterrupt:
    display(triangles_to_image(hof[0]))

In [ ]:
#triangles_to_image(hof[0]).save('sample_results/triangles-50-ssim.png')

In [ ]:
img1 = triangles_to_image(hof[0])
img2 = Image.open('sample_results/circle-30.png')

In [ ]:
img3 = np.clip((np.array(img1, dtype='float')+np.array(img2, dtype='float'))/2, 0, 255)
img3 = Image.fromarray(img3.astype('uint8'))

In [ ]:
img3